In [1]:
!pip install -q webdataset

In [9]:
import os
import json
import multiprocessing
from pathlib import Path

import polars as pl
import webdataset as wds
from tqdm import tqdm

from graphnet.data.constants import FEATURES, TRUTH


In [3]:
meta_data_path = Path("../../raw/icecube-neutrinos-in-deep-ice/train_meta.parquet")
geometry_path = Path("../../raw/icecube-neutrinos-in-deep-ice/sensor_geometry.csv")
input_data_path = "../../raw/icecube-neutrinos-in-deep-ice/train/batch_{batch_id}.parquet"
shard_dir = Path("./webdatasets")
shard_dir.mkdir(exist_ok=True)

shard_filename = str(shard_dir / 'shards-%04d.tar')

shard_size = int(1 * 1024**3)

In [4]:
meta_data = pl.read_parquet(meta_data_path)
geometry_table = pl.read_csv(geometry_path, dtypes={"sensor_id": pl.Int16})

In [5]:
def get_features_truth(event_id):
    truth = meta_data_batch.filter(pl.col("event_id") == event_id)[TRUTH.KAGGLE].to_numpy()
    features = (
        df_batch.filter(pl.col("event_id") == event_id)
            .join(geometry_table, on="sensor_id", how="left")[FEATURES.KAGGLE]
            .to_numpy()
    )
    return features, truth

In [6]:
batch_ids = range(51, 101)

with wds.ShardWriter(shard_filename, maxsize=shard_size,) as sink:
    for batch_id in batch_ids:
        meta_data_batch = meta_data.filter(pl.col("batch_id") == batch_id)
        event_ids = meta_data_batch["event_id"].unique()
        df_batch = pl.read_parquet(input_data_path.format(batch_id=batch_id))

        for event_id in tqdm(event_ids, desc="fBatch {batch_id}"):
            features, truth = get_features_truth(event_id)
            sink.write({
                "__key__": str(event_id),
                "pickle": (
                    features, truth,
                )
            })
            
        del df_batch, meta_data_batch
        break


# writing webdatasets/shards-0000.tar 0 0.0 GB 0


fBatch {batch_id}:  50%|██████████████████                  | 100007/200000 [52:06<55:56, 29.79it/s]

# writing webdatasets/shards-0001.tar 100000 0.8 GB 100000


fBatch {batch_id}: 100%|██████████████████████████████████| 200000/200000 [1:43:36<00:00, 32.17it/s]
